# Speed tests against a specific Colab GCE instance

This notebook runs a series of epidemic-spreading experiments using diffewrent network sizes and different target degrees of parallelism. The idea is to run the same experiments on different virtual machine instance types drawn from Google Compute Engine's repertoire to see hto what extent we can support compute-bound experiments.

In [ ]:
!pip install epydemic

In [ ]:
from epyc import Experiment, LabNotebook, Lab, ParallelLab
from epydemic import SIR, ERNetwork, StochasticDynamics

## Basic setup

In [ ]:
# create a notebook for all the results
nb = LabNotebook()

In [ ]:
# ER topology parameters
N_small = int(1e4)
N_large = int(1e5)
kmean = 10

# common parameters
params = dict()
params[ERNetwork.N] = N_small
params[ERNetwork.KMEAN] = kmean
params[SIR.P_INFECT] = 0.01
params[SIR.P_INFECTED] = 0.001
params[SIR.P_REMOVE] = 0.005

## Single-core performance

We run one experiment within a sequential lab to obtain the single-threaded (andf therefore single-core) performance, doing 10 repetitions of the experiment to and then averaging.

In [ ]:
# create the sequential lab
lab = Lab(nb)

In [ ]:
# copy in the ]parameters
for k in params:
  lab[k] = params[k]

In [ ]:
# run the experiments
lab['cores'] = 1
lab['repetitions'] = range(10)
e = StochasticDynamics(SIR(), ERNetwork())
lab.runExperiment(e)

In [ ]:
# extract the results and summary statistics
df = nb.dataframe()
experiments = df[(df[ERNetwork.N] == N_small) & (df['cores'] == 1)]
t_total = (max(experiments[Experiment.END_TIME]) - min(experiments[Experiment.START_TIME])).seconds
ts = experiments[Experiment.ELAPSED_TIME]
print("Small network, single core; elapsed time {:d}s, mean {:.2f}s, variance {:.2f}s".format(t_total, ts.mean(), ts.std()))

Small network, single core; elapsed time 74s, mean 7.44s, variance 0.23s


We can then try the same experiment on larger network.

In [ ]:
lab[ERNetwork.N] = N_large
lab.runExperiment(e)

In [ ]:
df = nb.dataframe()
experiments = df[(df[ERNetwork.N] == N_large) & (df['cores'] == 1)]
t_total = (max(experiments[Experiment.END_TIME]) - min(experiments[Experiment.START_TIME])).seconds
ts = experiments[Experiment.ELAPSED_TIME]
print("Large network, elapsed time {:d}s, mean {:.2f}s, variance {:.2f}s".format(t_total, ts.mean(), ts.std()))

Large network, elapsed time 930s, mean 93.09s, variance 0.56s


## Multicore performance

We now test on multiple cores, using all the cores that the machine reports itself to have and running 10 repetitions for each core.

In [ ]:
# create the parallel lab
plab = ParallelLab(notebook=nb)
ncores = plab.numberOfCores()

In [ ]:
# copy over the basic parameters
for k in params:
  plab[k] = params[k]

In [ ]:
plab['cores'] = ncores
plab['repetitions'] = range(10 * ncores)
e = StochasticDynamics(SIR(), ERNetwork())
plab.runExperiment(e)

In [ ]:
df = nb.dataframe()
experiments = df[(df[ERNetwork.N] == N_small) & (df['cores'] == ncores)]
t_total = (max(experiments[Experiment.END_TIME]) - min(experiments[Experiment.START_TIME])).seconds
ts = experiments[Experiment.ELAPSED_TIME]
print("Small network, {:d} cores, elapsed time {:d}s, mean {:.2f}s, variance {:.2f}s".format(ncores, t_total, ts.mean(), ts.std()))

Small network, 4 cores, elapsed time 69s, mean 6.83s, variance 0.08s


And then again on the larger network.

In [ ]:
plab[ERNetwork.N] = N_large
plab.runExperiment(e)

In [ ]:
df = nb.dataframe()
experiments = df[(df[ERNetwork.N] == N_large) & (df['cores'] == ncores)]
t_total = (max(experiments[Experiment.END_TIME]) - min(experiments[Experiment.START_TIME])).seconds
ts = experiments[Experiment.ELAPSED_TIME]
print("Large network, {:d} cores, elapsed time {:d}s, mean {:.2f}s, variance {:.2f}s".format(ncores, t_total, ts.mean(), ts.std()))

Large network, 4 cores, elapsed time 869s, mean 86.51s, variance 1.06s
